# Biography Generator - Single Cell Version

This notebook generates professional narratives, obituaries, or dating app introductions from uploaded documents using Claude AI.

**Just run the cell below and follow the prompts!**

**Features:**
- Load PDF or TXT documents
- Generate three types of biographies
- Iterative refinement with conversation history
- Export conversation history showing how outputs become inputs

In [ ]:
# ============================================================================
# BIOGRAPHY GENERATOR - ALL-IN-ONE CELL
# Just run this cell and follow the prompts!
# ============================================================================

# Install packages (only runs if not already installed)
try:
    import anthropic
    from pypdf import PdfReader
except ImportError:
    print("Installing required packages...")
    import sys
    !{sys.executable} -m pip install -q anthropic pypdf
    import anthropic
    from pypdf import PdfReader

import os
from datetime import datetime
import glob
from IPython.display import display, Markdown

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def extract_text_from_pdf(file_path):
    """Extract text from a PDF file."""
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

def read_text_file(file_path):
    """Read text from a text file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error reading text file: {e}")
        return None

# ============================================================================
# BIOGRAPHY GENERATOR CLASS
# ============================================================================

class BiographyGenerator:
    """Manages document loading and biography generation with conversation history."""
    
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.conversation_history = []  # Stores all messages for context
        self.document_content = None
        
    def load_document(self, file_path):
        """Load document content from a file."""
        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            return False
            
        if file_path.lower().endswith('.pdf'):
            self.document_content = extract_text_from_pdf(file_path)
        elif file_path.lower().endswith(('.txt', '.text')):
            self.document_content = read_text_file(file_path)
        else:
            print("❌ Unsupported file type. Please use .pdf or .txt files.")
            return False
            
        if self.document_content:
            print(f"✅ Document loaded successfully!")
            print(f"   Characters: {len(self.document_content)}")
            print(f"   Words: ~{len(self.document_content.split())}")
            return True
        return False
    
    def generate_content(self, output_type):
        if not self.document_content:
            return "❌ Please load a document first!"
        
        prompts = {
            1: """Based on the following information about a person, please write a professional narrative biography. 
This should be suitable for a professional website, LinkedIn profile, or corporate bio. 
Focus on career achievements, expertise, education, and professional impact.
Make it compelling but formal and professional.

Document content:
{content}

Generate a well-structured professional narrative (approximately 150-300 words).""",
            
            2: """Based on the following information about a person, please write a thoughtful and respectful obituary.
Include key life events, achievements, family connections, and what made them special.
Strike a balance between celebrating their life and acknowledging the loss.

Document content:
{content}

Generate a well-structured obituary (approximately 200-400 words).""",
            
            3: """Based on the following information about a person, please write an engaging dating app introduction.
Make it authentic, warm, and interesting. Highlight personality, interests, values, and what makes them unique.
Use a conversational, friendly tone. Be genuine and not overly formal.
Include what they're looking for in a partner if that information is available.

Document content:
{content}

Generate a compelling dating app bio (approximately 100-200 words)."""
        }
        
        prompt = prompts[output_type].format(content=self.document_content)
        self.conversation_history = [{"role": "user", "content": prompt}]
        
        print("🤔 Generating content...\n")
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        assistant_message = response.content[0].text
        self.conversation_history.append({"role": "assistant", "content": assistant_message})
        return assistant_message
    
    def follow_up(self, user_message):
        if not self.conversation_history:
            return "❌ Please generate content first!"
        
        self.conversation_history.append({"role": "user", "content": user_message})
        
        print("🤔 Processing your request...\n")
        response = self.client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=2000,
            messages=self.conversation_history
        )
        
        assistant_message = response.content[0].text
        self.conversation_history.append({"role": "assistant", "content": assistant_message})
        return assistant_message
    
    def export_conversation(self, filename=None):
        if not self.conversation_history:
            print("❌ No conversation to export!")
            return None
        
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"conversation_history_{timestamp}.txt"
        
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write("="*80 + "\n")
                f.write("BIOGRAPHY GENERATOR - CONVERSATION HISTORY\n")
                f.write("="*80 + "\n")
                f.write(f"Exported: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Total Messages: {len(self.conversation_history)}\n")
                f.write("="*80 + "\n\n")
                
                for i, message in enumerate(self.conversation_history, 1):
                    role = "USER" if message["role"] == "user" else "CLAUDE"
                    f.write(f"\n{'='*80}\nMESSAGE {i}: {role}\n{'='*80}\n\n")
                    f.write(message["content"] + "\n\n")
            return filename
        except Exception as e:
            print(f"❌ Error exporting conversation: {e}")
            return None

# ============================================================================
# MAIN PROGRAM
# ============================================================================

print("\n" + "="*70)
print("          📝 BIOGRAPHY GENERATOR 📝")
print("="*70 + "\n")

# Get API key
API_KEY = os.environ.get('ANTHROPIC_API_KEY')
if not API_KEY:
    API_KEY = input("Enter your Anthropic API key: ").strip()

generator = BiographyGenerator(API_KEY)
print("✅ Generator initialized!\n")

# Browse files
pdf_files = glob.glob("*.pdf")
txt_files = glob.glob("*.txt")
all_files = sorted(set(pdf_files + txt_files))

print("📁 Available files:\n")
for i, file in enumerate(all_files, 1):
    file_size = os.path.getsize(file) / 1024
    file_type = "PDF" if file.endswith('.pdf') else "TXT"
    print(f"  {i}. {file:<50} ({file_size:.1f} KB) [{file_type}]")

if all_files:
    file_choice = input(f"\nSelect file number (1-{len(all_files)}), or enter file path: ").strip()
    
    if file_choice.isdigit() and 1 <= int(file_choice) <= len(all_files):
        file_path = all_files[int(file_choice) - 1]
    else:
        file_path = file_choice
else:
    file_path = input("Enter file path: ").strip()

# Load document
if generator.load_document(file_path):
    # Select output type
    print("\n📋 Select Output Type:")
    print("  1. Professional Narrative")
    print("  2. Obituary")
    print("  3. Dating App Introduction")
    
    output_type = int(input("\nEnter choice (1-3): ").strip())
    
    type_names = {
        1: "Professional Narrative",
        2: "Obituary",
        3: "Dating App Introduction"
    }
    
    # Generate content
    result = generator.generate_content(output_type)
    
    print("\n" + "="*70)
    print(f"  {type_names[output_type].upper()}")
    print("="*70)
    print(result)
    print("="*70 + "\n")
    
    # Multiple refinements loop
    while True:
        refine = input("\nWould you like to refine the content? (y/n): ").strip().lower()
        
        if refine == 'y':
            refinement = input("How would you like to refine it? (e.g., 'Make it shorter'): ").strip()
            result = generator.follow_up(refinement)
            
            print("\n" + "="*70)
            print(f"  REFINED {type_names[output_type].upper()}")
            print("="*70)
            print(result)
            print("="*70 + "\n")
        else:
            break
    
    # Save option
    save = input("\nSave result to file? (y/n): ").strip().lower()
    if save == 'y':
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = f"{type_names[output_type].lower().replace(' ', '_')}_{timestamp}.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"{type_names[output_type].upper()}\n{'='*70}\n\n{result}\n")
        print(f"✅ Saved to: {output_file}")
    
    # Export conversation
    export = input("\nExport conversation history? (y/n): ").strip().lower()
    if export == 'y':
        filename = generator.export_conversation()
        if filename:
            print(f"✅ Conversation exported to: {filename}")

print("\n👋 Done!")

## Key Concepts Explained

### 1. Conversation History Management
The `conversation_history` list stores all messages (both user and assistant). Each time you call Claude's API, you send the **entire history**, which allows Claude to remember what it said before.

### 2. Output as Input
When you ask Claude to refine content, the previous response is already in the conversation history, so Claude can modify its own earlier output.

### 3. Document Processing
The code extracts text from PDFs and TXT files, converting them into a format that Claude can understand.

### 4. Prompt Engineering
Different prompts produce different styles of output (professional, obituary, dating app bio).